# 2주차 과제 제출에 포함되어야 하는 형태 (최소)
1. 모델링에 대한 결과 .ipynb 파일 (team1_onboarding_week2_analysis.ipynb)
2. np.random.seed(42) 설정
3. Input : X데이터 : [user_id, +@]
4. Output : Count column 예측값
5. 2020.01.01~2020.09.30 : Training Data
6. 2020.10.01~2020.12.31 : Test Data
7. 2020.10.01~2020.12.31 구간에 대한 MSE, MAE 계산값
8. 요일별 이용량에 대한 분석 - EDA
9. 유저별 이용 count 값에 대한 분석 - EDA

In [1]:
import os 
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'  # (Windows 용) 한글 출력을 위한 글꼴 설정

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings("ignore")

# 랜덤시드 통일
np.random.seed(42)

In [2]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    # colab 사용자는 system_name이 'Linux'로 확인

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux
        # colab에서는 runtime을 <꼭> 재시작 해야함.
        # 런타임을 재시작 하지 않고 폰트 설치를 하면 기본 설정 폰트가 로드되어 한글이 깨짐.
        !apt-get update -qq
        !apt-get install fonts-nanum -qq  > /dev/null

        import matplotlib.font_manager as fm

        fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
        font = fm.FontProperties(fname=fontpath, size=9)
        fm._rebuild()
        font_family = "NanumBarunGothic"
    return font_family

In [3]:
# 시각화를 위한 폰트설정
# 위에서 만든 함수를 통해 시스템 폰트를 불러와서 font_family 라는 변수에 할당.
a = get_font_family()
# 폰트설정
import matplotlib.pyplot as plt 
plt.rc("font", family = a)
# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)
# ggplot으로 그래프 스타일 설정 / 개인 자유
plt.style.use("ggplot")

In [4]:
# 데이터 확인
os.listdir('data')

['2020교통량통합.xlsx',
 'check.csv',
 'final.csv',
 'holiday.csv',
 'metro.csv',
 '교통량정리.csv',
 '국가공휴일.xlsx',
 '기상청.csv',
 '디지털 스킬셋 기술과제.docx',
 '서울시_기상데이터.csv',
 '실전db.csv',
 '실전db_holiday.csv',
 '지하철노선위경도정보3.xlsx',
 '최종.csv']

In [5]:
# 데이터프레임 불러오기
df = pd.read_csv("./data/최종.csv")
df = df.drop("Unnamed: 0",axis=1)
df = df.sort_values(by="DATE")

In [6]:
X_train = df[:623305]
X_test = df[623305:]

In [7]:
train_x = X_train.drop(["DATE", "COUNT"], axis=1)
train_y = X_train['COUNT']

In [8]:
test_x = X_test.drop(["DATE", "COUNT"], axis=1)
real_count = X_test['COUNT']

In [9]:
random_state = 42

gboost = GradientBoostingRegressor(random_state=random_state)
xgboost = XGBRegressor(random_state=random_state)
lightgbm = LGBMRegressor(random_state=random_state)
rdforest = RandomForestRegressor(random_state=random_state)

models = [gboost, xgboost, lightgbm, rdforest]

In [10]:
def get_scores(models, train_x, train_y, test_x):
    df = {}

    for model in models:      
        model.fit(train_x, train_y)
        y_pred = model.predict(test_x)
        print("사용한 모델 :",model)
        print("MSE :",mean_squared_error(real_count, y_pred))
        print()
        print()

In [11]:
get_scores(models, train_x, train_y, test_x)

사용한 모델 : GradientBoostingRegressor(random_state=42)
MSE : 0.03795805688767001


사용한 모델 : XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
MSE : 0.03869691377648899


사용한 모델 : LGBMRegressor(random_state=42)
MSE : 0.03599285384130974


사용한 모델 : RandomForestRegressor(random_state=42)
MSE : 0.046690254564686534




In [20]:
# lightgbm이 가장 낮다!!
# 해당 모델의 가장 좋은 파라미터를 찾아보자

from sklearn.model_selection import GridSearchCV

# 파라미터 범위
param_grid = {
    'n_estimators': [10, 30, 50],
    'max_depth': [10, 20],
    'learning_rate' : [0.05],
    'num_iterations' : [100,200],
}

# 모델 준비 (LGBMRegressor)
model = LGBMRegressor(random_state=random_state)

In [21]:
# 위 과정을 함수로 만들어 여러 모델에 시험해봅시다.

GCV = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=5)


# 모델 fitting
GCV.fit(train_x, train_y)

# 가장 좋은 파라미터 찾기
print("Best Param :", GCV.best_params_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  1.4min
[Parallel(n_jobs=5)]: Done  60 out of  60 | elapsed:  2.1min finished


Best Param : {'learning_rate': 0.05, 'max_depth': 20, 'n_estimators': 10, 'num_iterations': 200}


In [22]:
# 가장 좋은 파라미터로 모델 설정
model=GCV.best_estimator_ 

# 예측
y_pred = model.predict(test_x)

In [23]:
print("MSE :",mean_squared_error(real_count, y_pred))

MSE : 0.03596898702809573
